# Testing Difference in Means
- Using `statsmodels.power.tt_ind_solve_power`
- https://www.statsmodels.org/dev/generated/statsmodels.stats.power.tt_ind_solve_power.html#statsmodels.stats.power.tt_ind_solve_power
- solve for any one parameter of the power of a two sample t-test

## Terminology
In `tt_ind_solve_power`, `effect_size` is the standardized effect size, also known as Cohen's d.

In [1]:
import numpy as np
from statsmodels.stats import power

In [2]:
m1 = 30  # Mean of group 1
m2 = 28  # Mean of group 2
s1 = 3  # Standard deviation of group 1
s2 = 3   # Standard deviation of group 2
n1 = 20  # Sample size of group 1
n2 = 25  # Sample size of group 2

In [3]:
# Function to calculate pooled standard deviation
def pooled_std(s1, s2, n1, n2):
    return np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))

# statsmodels calls this the standardized effect size
# Function to calculate Cohen's d
def cohens_d(m1, m2, s1, s2, n1, n2):
    # Calculate the pooled standard deviation
    pooled_sd = pooled_std(s1, s2, n1, n2)
    
    # Calculate Cohen's d
    d = (m1 - m2) / pooled_sd
    return d

In [8]:
required_nobs_for_group_1 = power.tt_ind_solve_power(
    effect_size= cohens_d(m1, m2, s1, s2, n1, n2),
    alpha=0.05,
    power=0.85,
    ratio=1
)
required_nobs_for_group_1

41.386145352912585